In [23]:
#importing all the necessary packages
import pickle
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot
import numpy as np
from skimage import data
from skimage.transform import resize

In [24]:
#importing all the necessary packages
import keras
from keras import models
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import  SGD
from keras.utils import np_utils
from keras import backend as K
from keras.callbacks import ModelCheckpoint

In [12]:
#Change working directory
import os
os.getcwd()
os.chdir('/home/puneetj/Data_DL/')

In [13]:
#import pickled data
training_set = pickle.load(open("training_set.pkl", "rb"))

In [14]:
#separating classes and image matrix
#normalizing the value between 0 to 255
train_X = []
train_class = []
for image in training_set:
    pixel = image["img"]
    classes = image["class"]
    train_X.append(pixel)
    train_class.append(classes)

In [27]:
train_X[32]

array([[[ 0.38185412,  0.44826353,  0.49253647],
        [ 0.43745882,  0.49938824,  0.54181647],
        [ 0.44272941,  0.48225882,  0.50927059],
        ..., 
        [ 0.41110588,  0.46315294,  0.43548235],
        [ 0.50004706,  0.54761412,  0.46684235],
        [ 0.4704    ,  0.51467294,  0.41229176]],

       [[ 0.46486588,  0.53364706,  0.54695529],
        [ 0.52877804,  0.60405333,  0.60766118],
        [ 0.52665098,  0.59328627,  0.57935686],
        ..., 
        [ 0.49430588,  0.50610196,  0.47316078],
        [ 0.61886745,  0.62320941,  0.52109804],
        [ 0.59939765,  0.61441882,  0.48397176]],

       [[ 0.46974118,  0.52508235,  0.49741176],
        [ 0.5347451 ,  0.61204706,  0.56727843],
        [ 0.5225098 ,  0.6147451 ,  0.55466667],
        ..., 
        [ 0.53270588,  0.47890196,  0.45129412],
        [ 0.68517647,  0.62039216,  0.52131765],
        [ 0.67661176,  0.64103529,  0.51190588]],

       ..., 
       [[ 0.23849412,  0.27604706,  0.22795294],
        

In [16]:
#resizing the pixels  to 50X50 matrix
for no,image in enumerate(train_X):
    train_X[no] = resize(image,(50,50,3))
    train_X[no].astype("float32")

In [17]:
#split between triain and validation
train = train_X[:27446]
train_clas = train_class[:27446]
validation = train_X[27446:]
validation_clas = train_class[27446:]

In [18]:
#convert to numpy array
train = np.array(train)
validation = np.array(validation)

In [43]:
#set the  object for number of classes and one hot encoding for the test and validation
n_classes=max(train_class)+1
train_clas = np_utils.to_categorical(train_clas, nb_classes=n_classes)
validation_clas = np_utils.to_categorical(validation_clas, nb_classes=n_classes)

#set shape according to dim ordering
if K.image_dim_ordering() == 'tf':
    train = train.reshape(train.shape[0],50, 50,3)
    validation = validation.reshape(validation.shape[0], 50, 50,3)
    input_shape = (50, 50,3)    
else:
    train = train.reshape(train.shape[0], 3, 50, 50)
    validation = validation.reshape(validation.shape[0], 3, 50, 50)
    input_shape = (3, 50, 50)

AttributeError: 'list' object has no attribute 'reshape'

In [ ]:
info_string = "50X50_Data_Test"
ckpt_fn = "/home/puneetj/Data_DL" + '{val_loss:.2f}-loss_' + info_string + '.h5'

ckpt = ModelCheckpoint(filepath=ckpt_fn,
                      monitor='val_loss')

In [30]:
#hyperparameteres
nb_filters1 = 32
nb_filters2 = 32
nb_filters3 = 64


# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)

model = Sequential()
#adding the convolution layer with btch normalizatioin and max pooling and drop-out
model.add(Convolution2D(nb_filters1, kernel_size[0], kernel_size[1],
                        border_mode='same',
                        input_shape=input_shape))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters2, kernel_size[0], kernel_size[1],border_mode='valid'))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

model.add(Convolution2D(nb_filters3, kernel_size[0], kernel_size[1],
                         border_mode = 'same'))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters3, kernel_size[0], kernel_size[1],border_mode='valid'))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

model.add(Convolution2D(nb_filters3, kernel_size[0], kernel_size[1],
                         border_mode = 'same'))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters3, kernel_size[0], kernel_size[1],border_mode='valid'))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))
#adding dense layer
model.add(Flatten())
model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))


model.add(Dense(n_classes))
model.add(BatchNormalization())
model.add(Activation('softmax'))

#compile model


model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=1e-08, decay=0.0),
              metrics=['accuracy'])
              
#run model
batch_size = 128
#using 6 epochs only, for runtime considerations
nb_epoch = 30


model.fit(train, train_clas, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(validation,validation_clas),callbacks=[ckpt],)

Train on 27446 samples, validate on 11763 samples
Epoch 1/30
27446/27446 [==============================] - 1180s - loss: 3.5783 - acc: 0.1040 - val_loss: 3.5583 - val_acc: 0.0993
Epoch 2/30
27446/27446 [==============================] - 1170s - loss: 2.7474 - acc: 0.3089 - val_loss: 2.7511 - val_acc: 0.3858
Epoch 3/30
27446/27446 [==============================] - 1164s - loss: 2.2503 - acc: 0.4686 - val_loss: 1.9627 - val_acc: 0.6057
Epoch 4/30
27446/27446 [==============================] - 1178s - loss: 1.9004 - acc: 0.5660 - val_loss: 1.4994 - val_acc: 0.7178
Epoch 5/30
27446/27446 [==============================] - 1180s - loss: 1.6235 - acc: 0.6449 - val_loss: 1.2169 - val_acc: 0.7676
Epoch 6/30
27446/27446 [==============================] - 1182s - loss: 1.4056 - acc: 0.6994 - val_loss: 1.0588 - val_acc: 0.8111
Epoch 7/30
27446/27446 [==============================] - 1174s - loss: 1.2278 - acc: 0.7431 - val_loss: 0.8547 - val_acc: 0.8471
Epoch 8/30
27446/27446 [================

In [33]:
#saving the model 
model.save('model_final.h5')

#evaluate the final model
score = model.evaluate(validation, validation_clas, verbose=1)
print('Test score:', score[0])

print('Test accuracy:', score[1])



11763/11763 [==============================] - 153s   
('Test score:', 0.11597815912822107)
('Test accuracy:', 0.97798180736206752)


12630/12630 [==============================] - 163s   


In [40]:
#scoring the test set
import csv 
validation= pickle.load(open("testing_set.pkl", "rb"))
validation_X = []
for image in validation:
    pixel = image["img"]
    validation_X.append(pixel)
for no,image in enumerate(validation_X):
    validation_X[no] = resize(image,(50,50,3))
    validation_X[no].astype("float32")
validation_X = np.array(validation_X)
#set shape according to dim ordering
if K.image_dim_ordering() == 'tf':
    train = train.reshape(train.shape[0],50, 50,3)
    validation_X = validation_X.reshape(validation_X.shape[0], 50, 50,3)
    input_shape = (50, 50,3)    
else:
    train = train.reshape(train.shape[0], 3, 50, 50)
    validation_X = validation_X.reshape(validation_X.shape[0], 3, 50, 50)
    input_shape = (3, 50, 50)

pred_val = model.predict_classes(validation_X,verbose=1)
csv_out = open("submission.csv",'wb')
mywriter = csv.writer(csv_out)
mywriter.writerow(pred_val.tolist())
csv_out.close()